# Overlapの計算テスト

In [20]:
import os 
import joblib
import numpy as np 
import matplotlib.pyplot as plt
from tqdm import tqdm
#plt.style.use("style.mpl")
# LaTeXフォントを有効にするための設定
plt.rcParams.update({
    "text.usetex": True,  # LaTeXを使用する
    "font.family": "serif",  # 使用するフォントファミリーを指定（serifやsans-serifなど）
    "font.serif": ["Times New Roman"],  # フォントの種類を指定（LaTeXがインストールされた環境で使用可能なもの）
    "text.latex.preamble": r"\usepackage{amsmath}",  # LaTeXプリアンブルにパッケージを追加する（必要に応じて）
})

In [32]:
class CFG:
    N= 100
    M= 6000
    L= 10
    senario = 'CL_fully'
    ini_type = "A"
    data_name= "MNIST"
    data_seed= 4244 #データのサンプリングを固定するためのseed
    seed1 = 820
    seed2 = 314
    seed3 = 1228
    layer_name_list= ['batch_normalization1', 'batch_normalization2', 'batch_normalization3','batch_normalization4', 'batch_normalization5',
                  'batch_normalization6', 'batch_normalization7', 'batch_normalization8','batch_normalization9','batch_normalization10',]

In [33]:
spinA = joblib.load(f"../../Result/CL_fully/Spin/spinA_{CFG.data_name}_ini{CFG.ini_type}_M{CFG.M}_L{CFG.L}_seed{CFG.seed1}.pkl")
spinB = joblib.load(f"../../Result/CL_fully/Spin/spinB_{CFG.data_name}_ini{CFG.ini_type}_M{CFG.M}_L{CFG.L}_seed{CFG.seed2}.pkl")

In [34]:
def calc_q_(A: np.ndarray, B: np.ndarray) -> float:
    M, N = A.shape
    dot_product = np.dot(A.T, B)
    x = np.sum(dot_product ** 2)
    x /= N * M * M
    x -= N / M
    return x

def calc_sim_q(A: np.ndarray, B: np.ndarray) -> float:
  mean = A * B
  sim_q = np.mean(mean)
  return sim_q



def get_q2(spinA, spinB):
    qab_dict={'time':spinA['time']}#時刻の初期化
    qaa_dict={'time':spinA['time']}
    q2_dict={'time':spinA['time']}
    for l in tqdm(CFG.layer_name_list):
        qab_list=[]
        qaa_list=[]
        q2_list=[]
        for i in range(len(spinA[l])):
            ab = calc_q_(spinA[l][i],spinB[l][i])
            aa= calc_q_(spinA[l][i],spinA[l][i])
            bb = calc_q_(spinB[l][i],spinB[l][i])
            q2 = ab/(np.sqrt(aa)*np.sqrt(bb))
            qab_list.append(ab)
            qaa_list.append(aa)
            q2_list.append(q2)
        qab_dict[l] = qab_list
        qaa_dict[l] = qaa_list
        q2_dict[l] = q2_list
    return qab_dict, qaa_dict, q2_dict

def get_normalized_spin(SpinA, SpinB):
  spinA_norm = SpinA.copy()
  spinB_norm = SpinB.copy()
  for l in tqdm(CFG.layer_name_list):
        squared_sum_A = np.sum(SpinA[l]**2, axis=2)
        squared_sum_B = np.sum(SpinB[l]**2, axis=2)
        # 規格化定数を計算
        normalization_constA = np.sqrt(100 / squared_sum_A)
        normalization_constB = np.sqrt(100 / squared_sum_B)
        # 規格化した配列を計算
        spinA_norm[l] = SpinA[l] * normalization_constA[:, :, np.newaxis]
        spinB_norm[l] = SpinB[l] * normalization_constB[:, :, np.newaxis]

  return spinA_norm, spinB_norm

In [35]:
spinA_norm, spinB_norm = get_normalized_spin(spinA,spinB)
qab, qaa, q2 = get_q2(spinA_norm,spinB_norm)

100%|██████████| 10/10 [00:07<00:00,  1.33it/s]


In [36]:
q2['batch_normalization1']

[0.519007451664905,
 0.5239856223190874,
 0.5237705676148959,
 0.5234551142484906,
 0.5232457910286241,
 0.5230858345705385,
 0.5229162536618087,
 0.52280836365051,
 0.5226272302255508,
 0.5224699865531209,
 0.5223502612531526,
 0.5221405906209071,
 0.5220595023764711,
 0.5220781316315442,
 0.5218535409868144,
 0.5217597114114833,
 0.5218167551092053,
 0.5217055540094013,
 0.5216688150270652,
 0.5216832797083758,
 0.5217402640239354,
 0.5215954349304536,
 0.5219027053738812,
 0.5220308160085895,
 0.5218998069311199,
 0.5219763872832648,
 0.5220858541432736,
 0.5220091411072021,
 0.5221108992676816,
 0.5219237564737903,
 0.5222640299151224,
 0.522215969918658,
 0.52210090228346,
 0.5224077276862741,
 0.5222082602234598,
 0.5224391983243427,
 0.5228881748720255,
 0.5227769928479654,
 0.5225624460459735,
 0.5224783637632583,
 0.5228157429485071,
 0.5231713962212285,
 0.5233035058175152,
 0.523193768295184,
 0.5232880162152885,
 0.5233168329538701,
 0.5229696829008984,
 0.5237852271912116,